# 大规模预训练的Transformer
:label:`sec_large-pretraining-transformers`

到目前为止，在我们的图像分类和机器翻译实验中，模型都是从头开始在包含输入-输出示例的数据集上训练以执行特定任务。例如，一个Transformer是在英语-法语对（:numref:`sec_transformer`）上训练的，以便该模型可以将输入的英文文本翻译成法文。因此，每个模型都成为了对数据分布的细微变化都很敏感的“特定专家”（:numref:`sec_environment-and-distribution-shift`）。为了获得更好的泛化模型，甚至是无需适应即可执行多项任务的更强大的“通才”，在大数据上进行*预训练*的模型变得越来越普遍。

给定更多的预训练数据，随着模型大小和训练计算量的增加，Transformer架构表现出了更好的*扩展*行为。具体来说，基于Transformer的语言模型的性能随着模型参数、训练标记数和训练计算量的数量呈幂律增长:cite:`kaplan2020scaling`。Transformer的可扩展性还体现在更大的视觉Transformer在更大数据上的显著提升性能（在:numref:`sec_vision-transformer`中讨论）。最近的成功案例包括Gato，这是一种可以玩Atari游戏、为图片添加字幕、聊天以及充当机器人:cite:`reed2022generalist`的“通才”模型。Gato是一个单一的Transformer，当在多种模态（包括文本、图像、关节扭矩和按钮按下）上进行预训练时，其扩展性很好。值得注意的是，所有这些多模态数据都被序列化为一个扁平的标记序列，这些标记序列可以像文本标记（:numref:`sec_transformer`）或图像块（:numref:`sec_vision-transformer`）一样被Transformer处理。

在Transformer成功地用于多模态数据预训练之前，它们已经广泛地用大量的文本进行了预训练。最初提出的用于机器翻译的Transformer架构（:numref:`fig_transformer`）由表示输入序列的编码器和生成目标序列的解码器组成。主要地，Transformer可以在三种不同的模式下使用：*仅编码器*、*编码器-解码器*和*仅解码器*。为了结束本章，我们将回顾这三种模式并解释Transformer预训练中的可扩展性。

## 仅编码器

当只使用Transformer编码器时，一串输入标记被转换成相同数量的表示，这些表示可以进一步投影到输出（例如，分类）。Transformer编码器由自注意力层组成，其中所有的输入标记相互关注。例如，如:numref:`fig_vit`所示的视觉Transformer是仅编码器的，它将一系列输入图像块转换为代表特殊“&lt;cls&gt;”标记的表示。由于这个表示依赖于所有输入标记，它会被进一步投影到分类标签。这种设计受到了早期仅编码器Transformer在文本上预训练的启发：BERT（双向编码器表示来自Transformers）:cite:`Devlin.Chang.Lee.ea.2018`。

### 预训练BERT

![左：通过掩码语言建模预训练BERT。“love”标记的预测取决于前后所有的输入标记。右：Transformer编码器中的注意力模式。垂直轴上的每个标记都关注水平轴上的所有输入标记。](../img/bert-encoder-only.svg)
:label:`fig_bert-encoder-only`

BERT是在文本序列上使用*掩码语言建模*进行预训练的：带有随机掩码标记的输入文本被送入Transformer编码器以预测被掩码的标记。如:numref:`fig_bert-encoder-only`所示，原始文本序列“I”，“love”，“this”，“red”，“car”前面加上了“&lt;cls&gt;”标记，并且“&lt;mask&gt;”标记随机替换了“love”；然后，在预训练过程中最小化被掩码的标记“love”与其预测之间的交叉熵损失。注意，在Transformer编码器的注意力模式（:numref:`fig_bert-encoder-only`右侧）中没有限制，因此所有标记都可以相互关注。因此，“love”的预测取决于序列中前后的输入标记。这就是为什么BERT被称为“双向编码器”。无需手动标注，书籍和维基百科的大规模文本数据可用于预训练BERT。

### 微调BERT

预训练的BERT可以通过*微调*来适应涉及单个文本或文本对的下游编码任务。在微调期间，可以向BERT添加具有随机参数的额外层：这些参数和预训练的BERT参数将*更新*以适应下游任务的训练数据。

![微调BERT进行情感分析。](../img/bert-finetune-classification.svg)
:label:`fig_bert-finetune-classification`

:numref:`fig_bert-finetune-classification`展示了微调BERT进行情感分析的情况。Transformer编码器是一个预训练的BERT，它接受文本序列作为输入，并将“&lt;cls&gt;”表示（输入的全局表示）传递给一个额外的全连接层以预测情感。在微调过程中，通过基于梯度的算法最小化情感分析数据上预测与标签之间的交叉熵损失，其中额外层是从零开始训练的，而预训练的BERT参数则被更新。BERT不仅仅能做情感分析。从2500亿个训练标记中学习到的一般语言表示的3.5亿参数的BERT，在自然语言任务上达到了先进水平，如单文本分类、文本对分类或回归、文本标注和问答。

您可能会注意到，这些下游任务包括文本对理解。BERT预训练还有一个损失函数用于预测一个句子是否直接跟随另一个句子。然而，后来发现当在20000亿个标记上预训练RoBERTa——一个同样大小的BERT变体时，这种损失不太有用:cite:`Liu.Ott.Goyal.ea.2019`。其他BERT的衍生版本改进了模型架构或预训练目标，例如ALBERT（强制参数共享）:cite:`lan2019albert`、SpanBERT（表示和预测文本片段）:cite:`joshi2020spanbert`、DistilBERT（通过知识蒸馏轻量化）:cite:`sanh2019distilbert`和ELECTRA（替换标记检测）:cite:`clark2019electra`。此外，BERT启发了计算机视觉中的Transformer预训练，例如视觉Transformer:cite:`Dosovitskiy.Beyer.Kolesnikov.ea.2021`、Swin Transformers:cite:`liu2021swin`和MAE（掩码自动编码器）:cite:`he2022masked`。

## 编码器-解码器

由于Transformer编码器将一串输入标记转换为相同数量的输出表示，因此仅编码器模式不能像在机器翻译中那样生成任意长度的序列。正如最初为机器翻译所提出的，Transformer架构可以配备一个解码器，该解码器自回归地预测任意长度的目标序列，逐标记地预测，条件是编码器输出和解码器输出：(i) 对于基于编码器输出的条件，编码器-解码器交叉注意力（:numref:`fig_transformer`中的解码器多头注意力）允许目标标记关注*所有*输入标记；(ii) 基于解码器输出的条件是通过所谓的*因果*注意力模式实现的（尽管这一名称在文献中很常见，但它与因果关系的研究几乎没有联系）（:numref:`fig_transformer`中的解码器掩码多头注意力），其中任何目标标记只能关注目标序列中的*过去*和*当前*标记。

要超越人工标注的机器翻译数据预训练编码器-解码器Transformer，BART :cite:`lewis2019bart` 和 T5 :cite:`raffel2020exploring` 是两个同时提出的预训练于大规模文本语料库的编码器-解码器Transformer。两者都在预训练目标中尝试重建原始文本，前者强调输入噪声化（例如，掩码、删除、置换和旋转），后者则突出多任务统一，并进行了全面的消融研究。

### 预训练T5

作为预训练的Transformer编码器-解码器的一个例子，T5（Text-to-Text Transfer Transformer）将许多任务统一为相同的文本到文本问题：对于任何任务，编码器的输入是一个任务描述（例如，“Summarize”, “:”）后跟任务输入（例如，文章中的一系列标记），解码器预测任务输出（例如，总结输入文章的一系列标记）。为了实现文本到文本的功能，T5训练生成一些目标文本，这些文本是基于输入文本的。

![左：通过预测连续跨度预训练T5。原句是“I”，“love”，“this”，“red”，“car”，其中“love”被替换为特殊的“&lt;X&gt;”标记，连续的“red”，“car”被替换为特殊的“&lt;Y&gt;”标记。目标序列以特殊“&lt;Z&gt;”标记结尾。右：Transformer编码器-解码器中的注意力模式。在编码器自注意力（下方正方形）中，所有输入标记相互关注；在编码器-解码器交叉注意力（上方矩形）中，每个目标标记关注所有输入标记；在解码器自注意力（上方三角形）中，每个目标标记仅关注当前和过去的自身标记（因果）。](../img/t5-encoder-decoder.svg)
:label:`fig_t5-encoder-decoder`

为了从任何原始文本中获取输入和输出，T5预训练以预测连续跨度。具体而言，文本中的标记被随机替换为特殊标记，其中每个连续跨度被替换为相同的特殊标记。考虑:numref:`fig_t5-encoder-decoder`中的例子，原文是“I”，“love”，“this”，“red”，“car”。标记“love”，“red”，“car”被随机替换为特殊标记。因为“red”和“car”是连续的跨度，所以它们被替换为相同的特殊标记。结果，输入序列为“I”，“&lt;X&gt;”，“this”，“&lt;Y&gt;”，目标序列为“&lt;X&gt;”，“love”，“&lt;Y&gt;”，“red”，“car”，“&lt;Z&gt;”，其中“&lt;Z&gt;”是另一个标记序列结束的特殊标记。如:numref:`fig_t5-encoder-decoder`所示，解码器具有因果注意力模式，以防止在序列预测过程中关注未来的标记。

在T5中，预测连续跨度也称为重构损坏的文本。通过这一目标，T5在C4（Colossal Clean Crawled Corpus）数据上进行了预训练，该数据包含从网络中提取的干净的英文文本:cite:`raffel2020exploring`，预训练使用的标记数量为1000亿。

### 微调T5

类似于BERT，T5需要在任务特定的训练数据上进行微调（更新T5参数）以执行此任务。与BERT微调的主要区别包括：(i) T5输入包括任务描述；(ii) T5可以利用其Transformer解码器生成任意长度的序列；(iii) 不需要额外的层。

![微调T5进行文本摘要。任务描述和文章标记都输入到Transformer编码器中以预测摘要。](../img/t5-finetune-summarization.svg)
:label:`fig_t5-finetune-summarization`

:numref:`fig_t5-finetune-summarization`解释了以文本摘要为例的T5微调过程。在这个下游任务中，任务描述标记“Summarize”，“:”后跟文章标记被输入到编码器中。

经过微调后，110亿参数的T5（T5-11B）在多个编码（例如，分类）和生成（例如，摘要）基准测试中取得了最先进的结果。自发布以来，T5已在后续研究中得到广泛应用。例如，基于T5设计的开关Transformer旨在激活一部分参数以提高计算效率:cite:`fedus2022switch`。在一个名为Imagen的文本到图像模型中，文本被输入到冻结的T5编码器（T5-XXL）中，该编码器拥有46亿参数:cite:`saharia2022photorealistic`。:numref:`fig_imagen`中的照片级真实感文本到图像示例表明，即使不进行微调，T5编码器本身也可以有效地表示文本。

![Imagen模型生成的文本到图像示例，其文本编码器来自T5（图取自:citet:`saharia2022photorealistic`）。](../img/imagen.png)
:width:`700px`
:label:`fig_imagen`


## 仅解码器

我们已经回顾了仅编码器和编码器-解码器Transformer。另一种选择是仅解码器Transformer，它去除了原始编码器-解码器架构（如:numref:`fig_transformer`所示）中的整个编码器和解码器子层中的编码器-解码器交叉注意力。如今，仅解码器Transformer已经成为大规模语言建模（:numref:`sec_language-model`）中的*事实标准*架构，它通过自我监督学习利用世界上丰富的未标记文本语料库。

### GPT和GPT-2

使用语言建模作为训练目标，GPT（生成预训练）模型选择了Transformer解码器作为其骨干架构:cite:`Radford.Narasimhan.Salimans.ea.2018`。

![左：使用语言建模预训练GPT。目标序列是输入序列向前移动一个标记。"&lt;bos&gt;"和"&lt;eos&gt;"是分别标记序列开头和结尾的特殊标记。右：Transformer解码器中的注意力模式。沿垂直轴的每个标记仅关注其沿水平轴的过去标记（因果）。](../img/gpt-decoder-only.svg)
:label:`fig_gpt-decoder-only`

按照:numref:`subsec_partitioning-seqs`中描述的自回归语言模型训练，:numref:`fig_gpt-decoder-only`说明了使用Transformer编码器进行GPT预训练的情况，其中目标序列是输入序列向前移动一个标记。请注意，Transformer解码器中的注意力模式强制每个标记只能关注其过去的标记（未来标记不能被关注，因为它们尚未被选择）。

GPT有1亿参数，需要针对各个下游任务进行微调。一年后引入了一个更大的Transformer解码器语言模型GPT-2:cite:`Radford.Wu.Child.ea.2019`。与GPT中的原始Transformer解码器相比，GPT-2采用了预规范化（在:numref:`subsec_vit-encoder`中讨论）、改进的初始化和权重缩放。在40GB文本上预训练的15亿参数GPT-2在语言建模基准测试中获得了最先进结果，并在多个其他任务上获得了令人鼓舞的结果*无需更新参数或架构*。

### GPT-3及以后

GPT-2展示了使用同一语言模型进行多个任务而无需更新模型的潜力。这种方法比微调更为计算高效，因为微调需要通过梯度计算来更新模型。

![使用语言模型（Transformer解码器）进行零样本、单样本、少样本上下文学习。不需要参数更新。](../img/gpt-3-xshot.svg)
:label:`fig_gpt-3-xshot`

在解释这种无需参数更新的语言模型更计算高效的使用方法之前，回想一下:numref:`sec_rnn-scratch`，语言模型可以被训练成根据某些前缀文本序列生成文本序列。因此，一个预训练的语言模型可以在*无需参数更新*的情况下生成任务输出，前提是输入序列包含了任务描述、任务特定的输入-输出示例和提示（任务输入）。这种学习范式被称为*上下文学习* :cite:`brown2020language`，可以根据是否有任务特定的输入-输出示例进一步分为*零样本*、*单样本*和*少样本*（:numref:`fig_gpt-3-xshot`）。

![GPT-3在所有42个准确性基准测试中的总体表现（标题改编自并采用:citet:`brown2020language`中的图表）。](../img/gpt3-xshot-scaling.png)
:width:`400px`
:label:`fig_gpt3-xshot-scaling`

这些设置在GPT-3 :cite:`brown2020language`中得到了测试，其最大版本使用的数据和模型规模比GPT-2大两个数量级。GPT-3使用与其直接前身GPT-2相同的Transformer解码器架构，只是交替层的注意力模式（:numref:`fig_gpt-decoder-only`右侧）更稀疏。通过3000亿个标记进行预训练，GPT-3的表现随模型规模增大而更好，其中少样本表现增长最快（:numref:`fig_gpt3-xshot-scaling`）。

随后的GPT-4模型在其报告中并未完全披露技术细节:cite:`openai2023gpt4`。与之前的模型相比，GPT-4是一个大规模的多模态模型，它可以接收文本和图像作为输入并生成文本输出。

## 可扩展性

:numref:`fig_gpt3-xshot-scaling`经验性地展示了GPT-3语言模型中Transformer的可扩展性。对于语言建模，更多关于Transformer可扩展性的综合实证研究让研究人员看到了希望，即通过更多的数据和计算来训练更大的Transformer :cite:`kaplan2020scaling`。

![随着模型规模（参数数量，不包括嵌入层）、数据集规模（训练标记数量）和训练计算量（PetaFLOP/s-day，不包括嵌入层）的增加，Transformer语言模型性能平稳提升。为了达到最佳性能，必须同步扩大这三个因素。当不受其他两个因素瓶颈限制时，经验性能与每个单独因素之间存在幂律关系（标题改编自并采用:citet:`kaplan2020scaling`中的图表）。](../img/scaling-power-law.png)
:width:`700px`
:label:`fig_scaling-power-law3`

如:numref:`fig_scaling-power-law3`所示，*幂律扩展*可以在性能方面观察到，这些性能与模型规模（参数数量，不包括嵌入层）、数据集规模（训练标记数量）和训练计算量（PetaFLOP/s-days，不包括嵌入层）有关。一般来说，同步增加所有这三个因素会导致更好的性能。但是，如何同步增加它们仍然是一个争论的话题 :cite:`hoffmann2022training`。

![Transformer语言模型训练运行情况（图取自:citet:`kaplan2020scaling`）。](../img/scaling-sample-conv.png)
:width:`700px`
:label:`fig_scaling-sample-conv`

除了性能提升外，大型模型还享有比小型模型更好的样本效率。:numref:`fig_scaling-sample-conv`显示，大型模型需要更少的训练样本（处理的标记数量）就能达到小型模型在同一水平上的性能，并且性能随着计算量的增加而平稳提升。

![GPT-3性能（交叉熵验证损失）随着用于训练的计算量遵循幂律趋势。:citet:`kaplan2020scaling`中观察到的幂律行为在另外两个数量级上仅有小偏差（标题改编自并采用:citet:`brown2020language`中的图表）。](../img/scaling-gpt3.png)
:width:`250px`
:label:`fig_scaling-gpt3`

在后续的大型Transformer模型中测试了:citet:`kaplan2020scaling`中的经验扩展行为。例如，GPT-3支持了这一假设，在:numref:`fig_scaling-gpt3`中又增加了两个数量级。

## 大型语言模型

GPT系列中Transformer的可扩展性激发了后续的大型语言模型。GPT-2 Transformer解码器被用于训练拥有5300亿参数的Megatron-Turing NLG :cite:`smith2022using`，使用了2700亿个训练标记。遵循GPT-2的设计，拥有2800亿参数的Gopher :cite:`rae2021scaling`在3000亿个训练标记上进行了预训练，在各种任务上表现出色。继承了同样的架构并使用了Gopher相同的计算预算，Chinchilla :cite:`hoffmann2022training`是一个大幅缩小（700亿参数）但训练时间更长（1.4万亿训练标记）的模型，在许多任务上优于Gopher，更加注重标记数量而非参数数量。为了继续扩展语言建模路线，PaLM（Pathway Language Model）:cite:`chowdhery2022palm`是一个5400亿参数的Transformer解码器，经过修改设计并在7800亿个标记上预训练，在BIG-Bench基准测试中超过了人类平均水平 :cite:`srivastava2022beyond`。它的后续版本PaLM 2 :cite:`anil2023palm`，数据和模型大致按1:1的比例扩展，提升了多语言和推理能力。其他大型语言模型，如进一步训练通用模型（PaLM）的Minerva :cite:`lewkowycz2022solving`和不在通用语料库上训练的Galactica :cite:`taylor2022galactica`，展示了有前途的定量和科学推理能力。

开源发布的模型，如OPT（Open Pretrained Transformers）:cite:`zhang2022opt`、BLOOM :cite:`scao2022bloom`和FALCON :cite:`penedo2023refinedweb`，民主化了大型语言模型的研究和应用。专注于推理时的计算效率，开源的Llama 1 :cite:`touvron2023llama`通过训练比通常使用的更多标记，超越了更大的模型。更新后的Llama 2 :cite:`touvron2023llama2`进一步将预训练语料库增加了40%，导致产品模型可能与竞争性的闭源模型相匹配。

:citet:`wei2022emergent`讨论了大型语言模型中出现的能力，这些能力在较大的模型中存在，但在较小的模型中不存在。然而，简单地增加模型大小并不必然使模型更好地遵循人类指令。:citet:`wei2021finetuned,sanh2021multitask`发现，通过对一系列通过*指令*描述的数据集进行微调，可以提高大型语言模型在保留任务上的零样本性能。使用*从人类反馈中强化学习*，:citet:`ouyang2022training`微调了GPT-3以遵循多样化的指令集。随之产生的InstructGPT通过微调将语言模型与人类意图对齐:cite:`ouyang2022training`，[ChatGPT](https://chat.openai.com/)能够基于与人类的对话生成类似人类的响应（例如代码调试和创意写作），并且可以在许多自然语言处理任务上进行零样本操作:cite:`qin2023chatgpt`。:citet:`bai2022constitutional`用模型输出部分自动化了指令调整过程，这也被称为*从AI反馈中强化学习*。

大型语言模型提供了一种令人兴奋的前景，即通过上下文学习（也称为*提示*）来制定文本输入，以诱导模型执行期望的任务。值得注意的是，*思维链提示* :cite:`wei2022chain`是一种上下文学习方法，它通过少量“问题、中间推理步骤、答案”示例来引发大型语言模型解决数学、常识和符号推理任务的复杂推理能力。采样多个推理路径 :cite:`wang2023self`、多样化少数示例 :cite:`zhang2023automatic`和将复杂问题简化为子问题 :cite:`zhou2023least`都可以提高推理准确性。实际上，只需简单的提示，比如在每个答案前说“让我们一步一步思考”，大型语言模型甚至可以进行*零样本*思维链推理，准确率相当不错 :cite:`kojima2022large`。即使是包含文本和图像的多模态输入，语言模型也能比仅使用文本输入时更准确地进行多模态思维链推理 :cite:`zhang2023multicot`。



## 总结与讨论

Transformer已经被预训练为仅编码器（例如BERT）、编码器-解码器（例如T5）和仅解码器（例如GPT系列）。预训练的模型可以通过模型更新（例如微调）或不更新（例如少量样本）来适应执行不同任务。Transformer的可扩展性表明，更好的性能受益于更大的模型、更多的训练数据和更多的训练计算。由于Transformer最初是为了文本数据而设计和预训练的，因此本节略微偏向自然语言处理。尽管如此，上述讨论的那些模型经常出现在近期的跨多种模态的模型中。例如，
(i) Chinchilla :cite:`hoffmann2022training` 进一步扩展到了Flamingo :cite:`alayrac2022flamingo`，这是一个用于少量样本学习的视觉语言模型；
(ii) GPT-2 :cite:`Radford.Wu.Child.ea.2019` 和视觉Transformer在CLIP（对比语言-图像预训练）:cite:`radford2021learning` 中编码文本和图像，其图像和文本嵌入后来被DALL-E 2 文本到图像系统 :cite:`ramesh2022hierarchical` 采用。虽然目前还没有对多模态预训练中Transformer可扩展性的系统研究，但一种名为Parti :cite:`yu2022scaling` 的全Transformer文本到图像模型显示了跨模态可扩展性的潜力：更大的Parti更擅长高保真图像生成和内容丰富的文本理解（:numref:`fig_parti`）。

![Parti模型不同尺寸（350M, 750M, 3B, 20B）生成的相同文本的图像示例（示例取自:citet:`yu2022scaling`）。](../img/parti.png)
:width:`700px`
:label:`fig_parti`




## 练习

1. 是否可以使用包含不同任务的小批量来微调T5？为什么可以或不可以？GPT-2呢？
1. 给定一个强大的语言模型，你能想到哪些应用场景？
1. 如果让你通过添加额外层来微调一个语言模型以执行文本分类，你会在哪里添加这些层？为什么？
1. 考虑序列到序列的问题（例如，机器翻译），其中在整个目标序列预测过程中输入序列总是可用的。仅使用解码器Transformer建模可能存在什么局限性？为什么？

[讨论](https://discuss.d2l.ai/t/9232)